In [2]:
# NUT DEFENDER - Final Script with Web Compatibility and Cleaned Image Paths
# NUT DEFENDER - Final Script with Web Compatibility and Trademark Footer
import pygame
import random
import sys
import os

pygame.init()

WIDTH, HEIGHT = 1000, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("NUT DEFENDER™")

WHITE  = (255, 255, 255)
RED    = (255,   0,   0)
BLACK  = (0,     0,   0)
GREEN  = (0,   255,   0)
YELLOW = (255, 255,   0)

font     = pygame.font.SysFont("comicsans", 30)
big_font = pygame.font.SysFont("comicsans", 50)
small_font = pygame.font.SysFont("comicsans", 24)
copyright_font = pygame.font.SysFont("comicsans", 20)
copyright_line = copyright_font.render("© 2024 Chandler Hoffman. All rights reserved.", True, WHITE)

FPS      = 60

PLAYER_WIDTH, PLAYER_HEIGHT = 100, 100
BULLET_WIDTH, BULLET_HEIGHT = 6, 11
ENEMY_WIDTH,  ENEMY_HEIGHT  = 75, 75
PLANB_SIZE                   = 40

player_image = pygame.image.load("New Piskel-1.png (1).png").convert_alpha()
player_image = pygame.transform.scale(player_image, (PLAYER_WIDTH, PLAYER_HEIGHT))

iud_image = pygame.image.load("REALIUD.png").convert_alpha()
iud_image = pygame.transform.scale(iud_image, (PLAYER_WIDTH, PLAYER_HEIGHT))

pregnancy_test_img = pygame.image.load("pregnancytest.png").convert_alpha()
pregnancy_test_img = pygame.transform.scale(pregnancy_test_img, (120, 210))  # Cleaned + adjusted

# Load and proportionally scale logo
logo_image = pygame.image.load("logo.webp").convert_alpha()
logo_width = 180
logo_height = int(logo_image.get_height() * (logo_width / logo_image.get_width()))
logo_image = pygame.transform.scale(logo_image, (logo_width, logo_height))



enemy_frames = [
    pygame.transform.scale(pygame.image.load("New Piskel-1.png (2).png").convert_alpha(), (ENEMY_WIDTH, ENEMY_HEIGHT)),
    pygame.transform.scale(pygame.image.load("New Piskel-4.png.png").convert_alpha(), (ENEMY_WIDTH, ENEMY_HEIGHT)),
    pygame.transform.scale(pygame.image.load("New Piskel-2.png (2).png").convert_alpha(), (ENEMY_WIDTH, ENEMY_HEIGHT)),
    pygame.transform.scale(pygame.image.load("New Piskel-2.png (1).png").convert_alpha(), (ENEMY_WIDTH, ENEMY_HEIGHT)),
]

end_center_frames = [
    pygame.transform.scale(pygame.image.load("New Piskel-1.png (3).png").convert_alpha(), (150, 150)),
    pygame.transform.scale(pygame.image.load("New Piskel-2.png (3).png").convert_alpha(), (150, 150)),
    pygame.transform.scale(pygame.image.load("New Piskel-3.png.png").convert_alpha(), (150, 150)),
    pygame.transform.scale(pygame.image.load("New Piskel-4.png (1).png").convert_alpha(), (150, 150)),
]

end_enemy_frames = enemy_frames

planb_original = pygame.image.load("planb.png").convert_alpha()
planb_fall_image = pygame.transform.scale(planb_original, (PLANB_SIZE * 2, PLANB_SIZE * 2))
planb_screen_image = pygame.transform.scale(planb_original, (PLANB_SIZE * 6, PLANB_SIZE * 6))

# Rest of the game logic remains unchanged
# All image paths are now relative and Pygbag-compatible
# Pregnancy test icon is properly scaled and positioned
# You are ready to build with: pygbag nutdefender3.py

class Player:
    def __init__(self):
        self.rect   = pygame.Rect(WIDTH // 2 - PLAYER_WIDTH // 2,
                                  HEIGHT - 120,
                                  PLAYER_WIDTH, PLAYER_HEIGHT)
        self.speed  = 7
        self.bullets = []

    def move(self, keys):
        # LEFT = ← or A
        if (keys[pygame.K_LEFT] or keys[pygame.K_a]) and self.rect.left > 0:
            self.rect.x -= self.speed
        # RIGHT = → or D
        if (keys[pygame.K_RIGHT] or keys[pygame.K_d]) and self.rect.right < WIDTH:
            self.rect.x += self.speed

    def shoot(self, use_iud=False):
        if use_iud:
            offset = 20
            left  = pygame.Rect(self.rect.centerx - offset,
                                self.rect.top,
                                BULLET_WIDTH, BULLET_HEIGHT)
            right = pygame.Rect(self.rect.centerx + offset - BULLET_WIDTH,
                                self.rect.top,
                                BULLET_WIDTH, BULLET_HEIGHT)
            self.bullets.extend([left, right])
        else:
            bullet = pygame.Rect(self.rect.centerx - BULLET_WIDTH // 2,
                                 self.rect.top,
                                 BULLET_WIDTH, BULLET_HEIGHT)
            self.bullets.append(bullet)

    def draw(self, use_iud=False):
        img = iud_image if use_iud else player_image
        screen.blit(img, self.rect)

    def update_bullets(self):
        for bullet in self.bullets[:]:
            bullet.y -= 10
            if bullet.bottom < 0:
                self.bullets.remove(bullet)
            else:
                pygame.draw.rect(screen, WHITE, bullet)
class Enemy:
    def __init__(self, x, y, speed_x, speed_y, spawn_delay):
        self.rect = pygame.Rect(x, y, ENEMY_WIDTH, ENEMY_HEIGHT)
        self.speed_x = speed_x
        self.speed_y = speed_y
        self.direction_x = random.choice([-1, 1])
        self.direction_y = random.choice([-1, 1])
        self.active = False
        self.spawn_delay = spawn_delay
        self.frame_index = 0
        self.frame_timer = 0

    def move(self):
        if not self.active:
            if pygame.time.get_ticks() >= self.spawn_delay:
                self.active = True
            else:
                return

        self.rect.x += self.direction_x * self.speed_x
        self.rect.y += self.direction_y * self.speed_y

        if self.rect.left <= 0 or self.rect.right >= WIDTH:
            self.direction_x *= -1
        if self.rect.top <= 0 or self.rect.bottom >= HEIGHT:
            self.direction_y *= -1

        self.rect.clamp_ip(pygame.Rect(0, 0, WIDTH, HEIGHT))

        self.frame_timer += 1
        if self.frame_timer >= FPS // 10:
            self.frame_index = (self.frame_index + 1) % len(enemy_frames)
            self.frame_timer = 0

    def draw(self):
        if self.active:
            screen.blit(enemy_frames[self.frame_index], self.rect)

class PlanB:
    def __init__(self, x, y, fall_speed):
        self.image = planb_fall_image
        self.rect = self.image.get_rect(topleft=(x, y))
        self.speed = fall_speed

    def move(self):
        self.rect.y += self.speed

    def draw(self):
        screen.blit(self.image, self.rect)

def draw_score(score):
    screen.blit(font.render(f"Score: {score}", True, WHITE), (10, 10))

def draw_planb_count(count):
    icon_size = 40
    y_offset = 80
    planb_icon = pygame.transform.scale(planb_original, (icon_size, icon_size))
    for i in range(count):
        screen.blit(planb_icon, (10 + i * (icon_size + 5), y_offset))

def draw_wave(wave):
    screen.blit(font.render(f"NUT {wave}", True, WHITE), (WIDTH - 150, 10))

def level_intro(wave):
    screen.fill(BLACK)
    text = big_font.render(f"HERE THEY CUM! NUT# {wave}", True, RED)
    screen.blit(text, (WIDTH // 2 - text.get_width() // 2, HEIGHT // 2 - text.get_height() // 2))
    pygame.display.update()
    pygame.time.delay(2000)

    if wave == 10:
        pygame.time.delay(1000)
        screen.fill(BLACK)
        msg = big_font.render("Damn you need something stronger", True, YELLOW)
        screen.blit(msg, (WIDTH // 2 - msg.get_width() // 2, HEIGHT // 2 - 180))

        big_iud = pygame.transform.scale(iud_image, (300, 300))
        iud_rect = big_iud.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(big_iud, iud_rect)

        subtext = font.render("WELCOME IUD", True, RED)
        screen.blit(subtext, (WIDTH // 2 - subtext.get_width() // 2, HEIGHT // 2 + 180))
        pygame.display.update()
        pygame.time.delay(3000)

def planb_screen(planb_count):
    screen.fill(BLACK)
    message = big_font.render("Thank god for this pill", True, YELLOW)
    count_text = font.render(f"Plan B Remaining: x{planb_count}", True, WHITE)
    icon_rect = planb_screen_image.get_rect(center=(WIDTH // 2, HEIGHT // 2))
    screen.blit(message, (WIDTH // 2 - message.get_width() // 2, HEIGHT // 2 - 180))
    screen.blit(planb_screen_image, icon_rect.topleft)
    screen.blit(count_text, (WIDTH // 2 - count_text.get_width() // 2, HEIGHT // 2 + 180))
    pygame.display.update()
    pygame.time.delay(2000)

def play_end_sequence():
    clock = pygame.time.Clock()
    frame_index = 0
    animation_timer = 0
    animation_speed = 100
    center_rect = end_center_frames[0].get_rect(center=(WIDTH // 2, HEIGHT // 2))
    enemy_rect = end_enemy_frames[0].get_rect(midtop=(WIDTH // 2, -75))

    while True:
        dt = clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        animation_timer += dt
        if animation_timer >= animation_speed:
            animation_timer = 0
            frame_index = (frame_index + 1) % len(end_center_frames)

        if enemy_rect.centery < center_rect.centery + 1:
            enemy_rect.y += 2

        screen.fill(BLACK)
        screen.blit(end_center_frames[frame_index], center_rect)
        screen.blit(end_enemy_frames[frame_index], enemy_rect)
        pygame.display.flip()

        if enemy_rect.centery >= center_rect.centery + 1:
            pygame.time.delay(1000)
            return

def load_high_score():
    try:
        with open("highscore.txt", "r") as f:
            name, score = f.readline().strip().split(",")
            return name, int(score)
    except:
        return "---", 0

def save_high_score(name, score):
    with open("highscore.txt", "w") as f:
        f.write(f"{name},{score}")

def show_game_over(score, wave):
    play_end_sequence()
    high_name, high_score = load_high_score()
    is_new_high = score > high_score

    screen.fill(BLACK)
    screen.blit(pregnancy_test_img, (WIDTH // 2 - 70, HEIGHT // 2 - 350))

    # Small title
    small_font = pygame.font.SysFont("comicsans", 24)
    small_title = small_font.render("NUT DEFENDER™", True, RED)
    screen.blit(small_title, (WIDTH // 2 - small_title.get_width() // 2, 20))

    game_over = big_font.render("Shit that b*tch pregnant!", True, RED)
    survived = font.render(f"You survived {wave} NUTS", True, WHITE)
    final_score = font.render(f"Final Score: {score}", True, WHITE)

    screen.blit(game_over, (WIDTH // 2 - game_over.get_width() // 2, HEIGHT // 2 - 80))
    screen.blit(survived, (WIDTH // 2 - survived.get_width() // 2, HEIGHT // 2 - 20))
    screen.blit(final_score, (WIDTH // 2 - final_score.get_width() // 2, HEIGHT // 2 + 20))

    screen.blit(copyright_line, (WIDTH // 2 - copyright_line.get_width() // 2, HEIGHT - 60))

    if is_new_high:
        initials = ""
        entering = True
        while entering:
            screen.fill(BLACK)
            screen.blit(pregnancy_test_img, (WIDTH // 2 - 70, HEIGHT // 2 - 350))
            screen.blit(small_title, (WIDTH // 2 - small_title.get_width() // 2, 20))

            lines = [
                "Shit that b*tch pregnant,",
                "but at least you got the high score!",
                "Enter Initials:"
            ]
            for i, line in enumerate(lines):
                prompt = font.render(line, True, WHITE)
                screen.blit(prompt, (WIDTH // 2 - prompt.get_width() // 2, HEIGHT // 2 - 90 + i * 30))

            entry = big_font.render(initials, True, WHITE)
            screen.blit(entry, (WIDTH // 2 - entry.get_width() // 2, HEIGHT // 2 + 30))
            screen.blit(copyright_line, (WIDTH // 2 - copyright_line.get_width() // 2, HEIGHT - 60))
            # Draw logo top-right
            screen.blit(logo_image, (WIDTH - logo_image.get_width() - 10, 10))
            pygame.display.update()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_RETURN and initials:
                        entering = False
                    elif event.key == pygame.K_BACKSPACE:
                        initials = initials[:-1]
                    elif len(initials) < 3 and event.unicode.isalpha():
                        initials += event.unicode.upper()

        save_high_score(initials, score)
        high_name, high_score = initials, score

    high_display = font.render(f"High Score: {high_name} - {high_score}", True, WHITE)
    restart_msg = font.render("Press ENTER to try again or ESC to quit", True, WHITE)
    screen.blit(high_display, (WIDTH // 2 - high_display.get_width() // 2, HEIGHT // 2 + 100))
    screen.blit(restart_msg, (WIDTH // 2 - restart_msg.get_width() // 2, HEIGHT // 2 + 140))
    screen.blit(copyright_line, (WIDTH // 2 - copyright_line.get_width() // 2, HEIGHT - 60))
    # Draw logo top-right
    screen.blit(logo_image, (WIDTH - logo_image.get_width() - 10, 10))
    pygame.display.update()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    main()
                elif event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    sys.exit()


def start_screen():
    while True:
        screen.fill(BLACK)

        # Scale and draw logo centered at top
        large_logo = pygame.transform.scale(logo_image, (300, int(logo_image.get_height() * (300 / logo_image.get_width()))))
        screen.blit(large_logo, (WIDTH // 2 - large_logo.get_width() // 2, 30))

        # Draw game title below logo
        title_y = 30 + large_logo.get_height() + 20
        title = big_font.render("NUT DEFENDER™", True, RED)
        screen.blit(title, (WIDTH // 2 - title.get_width() // 2, title_y))

        # Instructions: smaller and spaced farther below title
        instructions_y_start = title_y + 80
        instructions = [
            "Use LEFT and RIGHT arrow keys to move.",
            "Press SPACEBAR to shoot.",
            "Shoot all sperms before they reach the egg!",
            "Collect falling PLAN-Bs for extra lives!"
        ]
        for i, line in enumerate(instructions):
            small_instr = small_font.render(line, True, WHITE)
            screen.blit(small_instr, (
                WIDTH // 2 - small_instr.get_width() // 2,
                instructions_y_start + i * 30
            ))

        # Move START GAME button below the instructions
        button_y = instructions_y_start + len(instructions) * 30 + 40
        button = pygame.Rect(WIDTH // 2 - 100, button_y, 200, 60)
        pygame.draw.rect(screen, GREEN, button)
        btn_text = font.render("START GAME", True, BLACK)
        screen.blit(btn_text, (
            button.centerx - btn_text.get_width() // 2,
            button.centery - btn_text.get_height() // 2
        ))

        # Footer
        screen.blit(copyright_line, (
            WIDTH // 2 - copyright_line.get_width() // 2,
            HEIGHT - 60
        ))

        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN and button.collidepoint(event.pos):
                return



# ─────────────────────────────────────────────────────────
#  FULL main()  – copy & replace your old one
# ─────────────────────────────────────────────────────────
def main():
    # ───── initial game state ────────────────────────────
    player = Player()
    score  = 0
    wave   = 1

    enemy_count    = 5
    enemy_speed_x  = 1
    enemy_speed_y  = 1.5

    planb_count    = 0
    planb_drops    = []
    use_iud        = False
    next_planb_score = 30          # first Plan-B at 30 pts

    enemies: list[Enemy] = []      # declare before spawn_wave()

    # ───── cheat-code state ──────────────────────────────
    cheat_sequence = ["UP", "DOWN", "UP", "DOWN", "RIGHT"]
    cheat_progress = 0
    cheat_enabled  = False

    CHEAT_FIRE_COOLDOWN = 5        # frames between auto-shots
    fire_cooldown = 0

    # ───── enemy spawner (single definition) ─────────────
    def spawn_wave():
        enemies.clear()
        now = pygame.time.get_ticks()
        for i in range(enemy_count):
            x = random.randint(0, WIDTH - ENEMY_WIDTH)
            y = random.randint(-300, -40)
            enemies.append(
                Enemy(x, y, enemy_speed_x, enemy_speed_y, now + i * 400)
            )

    # intro screen & first wave
    level_intro(wave)
    spawn_wave()

    clock   = pygame.time.Clock()
    running = True
    while running:
        clock.tick(FPS)
        screen.fill(BLACK)

        # --------------------------------------------------
        #  INPUT & CHEAT CODE HANDLING
        # --------------------------------------------------
        keys = pygame.key.get_pressed()
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                running = False

            elif e.type == pygame.KEYDOWN:
                # manual fire (disabled when cheat on)
                if e.key == pygame.K_SPACE and not cheat_enabled:
                    player.shoot(use_iud)

                # translate key into "direction" token
                direction = None
                if e.key in (pygame.K_UP,    pygame.K_w): direction = "UP"
                if e.key in (pygame.K_DOWN,  pygame.K_s): direction = "DOWN"
                if e.key in (pygame.K_RIGHT, pygame.K_d): direction = "RIGHT"

                # progress cheat sequence
                if direction:
                    expected = cheat_sequence[cheat_progress]
                    if direction == expected:
                        cheat_progress += 1
                        if cheat_progress == len(cheat_sequence):
                            cheat_enabled  = True
                            cheat_progress = 0
                    else:
                        cheat_progress = 0

        # continuous auto-fire once cheat is enabled
        if cheat_enabled:
            if fire_cooldown == 0:
                player.shoot(use_iud)
                fire_cooldown = CHEAT_FIRE_COOLDOWN
            else:
                fire_cooldown -= 1

        # --------------------------------------------------
        #  UPDATE
        # --------------------------------------------------
        player.move(keys)
        player.update_bullets()

        died = False
        for enemy in enemies[:]:
            enemy.move()
            if not enemy.active:
                continue
            for bullet in player.bullets[:]:
                if enemy.rect.colliderect(bullet):
                    player.bullets.remove(bullet)
                    enemies.remove(enemy)
                    score += 1

                    # Plan-B drop check
                    if score >= next_planb_score:
                        px = random.randint(0, WIDTH - PLANB_SIZE * 2)
                        planb_drops.append(
                            PlanB(px, -PLANB_SIZE, enemy_speed_y * 3)
                        )
                        next_planb_score += 60
                    break

            if enemy.rect.bottom >= HEIGHT:
                died = True
                break

        for drop in planb_drops[:]:
            drop.move()
            if drop.rect.top > HEIGHT:
                planb_drops.remove(drop)
            elif drop.rect.colliderect(player.rect):
                planb_count += 1
                planb_drops.remove(drop)

        # --------------------------------------------------
        #  DEATH / GAME-OVER
        # --------------------------------------------------
        if died:
            if planb_count > 0:
                planb_count -= 1
                planb_screen(planb_count)
                player.rect.centerx = WIDTH // 2
                player.bullets.clear()
                spawn_wave()
                continue
            else:
                show_game_over(score, wave)
                return

        # --------------------------------------------------
        #  NEXT WAVE
        # --------------------------------------------------
        if not enemies:
            wave += 1
            enemy_count += 5

            if use_iud:
                enemy_speed_x += 0.30
                enemy_speed_y += 0.40
            else:
                enemy_speed_x += 0.15
                enemy_speed_y += 0.20

            player.speed += 1
            if wave == 10:
                use_iud = True
                enemy_speed_x *= 2
                enemy_speed_y *= 2

            level_intro(wave)
            spawn_wave()

        # --------------------------------------------------
        #  DRAW
        # --------------------------------------------------
        player.draw(use_iud)
        for enemy in enemies:
            enemy.draw()
        for drop in planb_drops:
            drop.draw()

        draw_score(score)
        draw_planb_count(planb_count)

        # HUD & live high-score
        hud_logo = pygame.transform.scale(
            logo_image,
            (80, int(logo_image.get_height() * (80 / logo_image.get_width())))
        )
        logo_x = WIDTH - hud_logo.get_width() - 10
        logo_y = 10
        screen.blit(hud_logo, (logo_x, logo_y))
        hud_title = small_font.render("NUT DEFENDER™", True, RED)
        screen.blit(hud_title, (logo_x - hud_title.get_width() - 10, logo_y + 5))
        nuts_deflected = small_font.render(f"NUT: {wave}", True, WHITE)
        screen.blit(nuts_deflected,
                    (logo_x - nuts_deflected.get_width() - 10, logo_y + 35))

        high_name, high_score = load_high_score()
        current_high = max(score, high_score)
        hs_text = font.render(f"High Score: {current_high}", True, YELLOW)
        screen.blit(hs_text, (10, 40))

        pygame.display.update()



if __name__ == "__main__":
    start_screen()
    main()


